The primary purpose of this notebook is to test the OpenEphys sorting "pipeline" with a neuropixel file.
Follow the instructions from [kilosort](https://kilosort.readthedocs.io/en/latest/README.html#instructions)
To run Kilosort on GPU (strongly recommended), remove the CPU version of PyTorch with pip uninstall torch and proceed to 8. Otherwise, stop here.

Install the GPU version of PyTorch (for CUDA 11.8) with pip3 install torch --index-url https://download.pytorch.org/whl/cu118.

In [ ]:
import spikeinterface.widgets as sw
from AODR_session_sorters import OpenEphysSessionSorter as OES
from pyramid import cli
import pandas as pd
import numpy as np
import sys, os
%matplotlib ipympl

Below, we define the relevant paths. You should change these to match the corresponding paths on your machine. If you encounter issues while loading/sorting the file, chances are that your computer is having a hard time accessing the files from cloud storage. If that is the case, you need to transfer your files to a local directory, and also save the files to a local directory.

In [ ]:
# Experiment directory top level - should contain the subfolders like ecodes, python, matlab.
expDir = "C:/Users/lt711/Documents/GitHub/Lab_Pipelines/experiments/aodr"
sessDir = "MrM_NP_1_2025-11-04_14-33-27" #"MrM_2025-09-26-Trimmed"
os.chdir(expDir) # regular python files will run from the first open folder in your tree, but notebooks will work from wherever the notebook is stored.
# Directory where the raw files are stored to be converted/sorted
dataSearchPath = "C:/NeuronalData/Raw/"
# Where the rules for ecodes are stored
pyramidSearchPath = expDir+"/ecodes"
# Conversion specifications
convertSpecs = expDir+"/AODR_experiment_neuropixel.yaml"
# Base directory to save the output files from pyramid (hdf5 files)
baseSaveDir = "C:/NeuronalData/Converted/"
# The Open Ephys experiment name
currentFile = "experiment1.nwb"
# Full directory to save the output files from pyramid (hdf5 files)
trialFileOutputName = baseSaveDir+sessDir+".hdf5"
# Directory to save the output files from sorting
sorted_out = 'C:/NeuronalData/Sorted/'+sessDir+"/"
 
sys.path.append(expDir+"/python") # to make sure pyramid can access the custom collectors/enhancers/functions?

Below, we run the sorter/analyzer. With ~4 channels of neural data and a 90 min recording, this should take between 10-20 minutes. The step_names variable defines the sequential processing steps that are defined as methods in AODR_session_sorters OpenEphysSessionSorter class. If you use the step "open_sigui", a gui window will open that allows to interact with the sorting results. Close the gui to continue to the next step and save the output.

In [ ]:
sorter = OES(session_dir=dataSearchPath+sessDir+"/",
             out_folder=sorted_out,
             stream_name = 'ProbeA-AP',
             sorter_name='kilosort4',       # If not using kilosort, this is the actual name of the sorter given to spikeinterface. In this case it controls how the probe and temporary binary data are stored
             step_names=[                           # A list of all the sequential steps to take (class methods)
                 'clean_tree',                      # This removes everything from the output folder!!!
                 'read_data',                       # Has to be included to do anything else
                 'bandpass',                        # Bandpass filter the data - this is the only preprocessing we do
                 'run_kilosort4_gui',               # Run the kilosort4 GUI which will output a phy folder with the results
                 'open_phy'])

In [ ]:
sorter = OES(session_dir=dataSearchPath+sessDir+"/", out_folder=sorted_out,
             step_names=['read_data'])

In [ ]:
sorter.open_phy(alt_path="C:/NeuronalData/Sorted/MrM_NP_1_2025-11-04_14-33-27/kilosort4/")

TODO: Now, run Pyramid on the corresponding output to create a TrialFile. Make sure that the phy_reader.params_file points to the appropriate directory based on those you have defined above.

In [ ]:
cli.main(["convert", 
        "--trial-file", trialFileOutputName, 
        "--search-path", pyramidSearchPath, 
        "--experiment", convertSpecs, 
        "--readers", 
        "ttl_reader.session_dir="+dataSearchPath+sessDir,
        "message_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_x_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_y_reader.session_dir="+dataSearchPath+sessDir,
        "pupil_reader.session_dir="+dataSearchPath+sessDir,
        "phy_reader.params_file="+"C:/NeuronalData/Sorted/MrM_NP_1_2025-11-04_14-33-27/kilosort4/params.py"])